# MICE : Iterative Imputer

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('50_Startups.csv')

In [3]:
df.head(3)

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39


In [4]:
df = ( pd.read_csv('50_Startups.csv')[['R&D Spend', 'Administration', 'Marketing Spend', 'Profit']]/10000).round(0)
np.random.seed(9)
df = df.sample(5)
df

,R&D Spend,Administration,Marketing Spend,Profit
21,8.0,15.0,30.0,11.0
37,4.0,5.0,20.0,9.0
2,15.0,10.0,41.0,19.0
14,12.0,16.0,26.0,13.0
44,2.0,15.0,3.0,7.0


```python
# Alternatively
df_ = np.round(pd.read_csv('50_Startups.csv')[['R&D Spend', 'Administration', 'Marketing Spend', 'Profit']]/10000)
np.random.seed(9)
df_ = df.sample(5)
df_
```

## NaN value import ( Manipulating the Data )

In [5]:
df = df.iloc[:, 0:-1]

In [6]:
df.iloc[1, 0] = np.NaN
df.iloc[3, 1] = np.NaN
df.iloc[-1, -1] = np.NaN

C:\Users\Mohd Rizwan\AppData\Local\Temp\ipykernel_18828\1989045429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[1, 0] = np.NaN
C:\Users\Mohd Rizwan\AppData\Local\Temp\ipykernel_18828\1989045429.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[3, 1] = np.NaN
C:\Users\Mohd Rizwan\AppData\Local\Temp\ipykernel_18828\1989045429.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[-1, -1] = np.NaN


In [7]:
df

,R&D Spend,Administration,Marketing Spend
21,8.0,15.0,30.0
37,NaN,5.0,20.0
2,15.0,10.0,41.0
14,12.0,NaN,26.0
44,2.0,15.0,NaN


## Step 1: Impute all missing values with mean

In [8]:
df0 = pd.DataFrame()
df0['R&D Spend'] = df['R&D Spend'].fillna(df['R&D Spend'].mean())
df0['Administration'] = df['Administration'].fillna(df['Administration'].mean())
df0['Marketing Spend'] = df['Marketing Spend'].fillna(df['Marketing Spend'].mean())

In [9]:
df0

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,9.25,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.25,26.00
44,2.00,15.00,29.25


## Step 2: Remove the column 1 imputed value (Left to Right)

In [10]:
df1 = df0.copy()
df1.iloc[1, 0] = np.NaN
df1

,R&D Spend,Administration,Marketing Spend
21,8.0,15.00,30.00
37,NaN,5.00,20.00
2,15.0,10.00,41.00
14,12.0,11.25,26.00
44,2.0,15.00,29.25


## Training Data in X (Training Input)


In [11]:
X = df1.iloc[[0,2,3,4], [1,2]]
X

,Administration,Marketing Spend
21,15.00,30.00
2,10.00,41.00
14,11.25,26.00
44,15.00,29.25


## Training Data in Y (Corresponding Output)

In [12]:
Y = df1.iloc[[0,2,3,4], [0]]
Y

,R&D Spend
21,8.0
2,15.0
14,12.0
44,2.0


## Step 3 - Predict missing value of column 1

In [13]:
lr = LinearRegression()
lr.fit(X,Y)
pred = lr.predict(df1.iloc[1,1:].values.reshape(1,2))
pred

C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[23.14158651]])

In [14]:
df1.iloc[1, 0] = pred.round(2)
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.25,26.00
44,2.00,15.00,29.25


## Step 4 - Remove the column 2 imputed value (Left to Right)

In [15]:
df1.iloc[3, 1] = np.NaN
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.0,30.00
37,23.14,5.0,20.00
2,15.00,10.0,41.00
14,12.00,NaN,26.00
44,2.00,15.0,29.25


## Training Data in X (Training Input) | Column 2


In [16]:
X = df1.iloc[[0,1,2,4], [0,2]]
X

,R&D Spend,Marketing Spend
21,8.00,30.00
37,23.14,20.00
2,15.00,41.00
44,2.00,29.25


## Training Data in Y (Corresponding Output) | Column 2


In [17]:
Y = df.iloc[[0,1,2,4], [1]]
Y

,Administration
21,15.0
37,5.0
2,10.0
44,15.0


## Step 5 - Predict missing value of column 2


In [18]:
lr = LinearRegression()
lr.fit(X, Y)
pred = lr.predict(df1.iloc[3, [0,2]].values.reshape(1,2))
pred

C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[11.06331285]])

In [19]:
df1.iloc[3, [1]] = pred.round(2)
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.06,26.00
44,2.00,15.00,29.25


## Step 6 - Remove the column 3 imputed value (Left to Right)

In [20]:
df1.iloc[-1, -1] = np.NaN
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.0
37,23.14,5.00,20.0
2,15.00,10.00,41.0
14,12.00,11.06,26.0
44,2.00,15.00,NaN


## Training Data in X (Training Input) | Column 3


In [21]:
X = df1.iloc[:-1, :-1]
X

,R&D Spend,Administration
21,8.00,15.00
37,23.14,5.00
2,15.00,10.00
14,12.00,11.06


## Training Data in Y (Corresponding Output) | Column 3


In [22]:
Y = df1.iloc[:-1, [-1]]
Y

,Marketing Spend
21,30.0
37,20.0
2,41.0
14,26.0


## Step 7 - Predict missing value of column 3


In [23]:
lr.fit(X, Y)
pred = lr.predict(df1.iloc[-1, :-1].values.reshape(1,2))
pred

C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[31.56351448]])

In [24]:
df1.iloc[-1, -1] = pred.round(2)
df1

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.06,26.00
44,2.00,15.00,31.56


## Step 8 - Subtract 0th (df0) iteration from 1st (df1) iteration

In [25]:
df1 - df0

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.00
37,13.89,0.00,0.00
2,0.00,0.00,0.00
14,0.00,-0.19,0.00
44,0.00,0.00,2.31


In [26]:
df0, df, df1

(    R&D Spend  Administration  Marketing Spend
 21       8.00           15.00            30.00
 37       9.25            5.00            20.00
 2       15.00           10.00            41.00
 14      12.00           11.25            26.00
 44       2.00           15.00            29.25,
     R&D Spend  Administration  Marketing Spend
 21        8.0            15.0             30.0
 37        NaN             5.0             20.0
 2        15.0            10.0             41.0
 14       12.0             NaN             26.0
 44        2.0            15.0              NaN,
     R&D Spend  Administration  Marketing Spend
 21       8.00           15.00            30.00
 37      23.14            5.00            20.00
 2       15.00           10.00            41.00
 14      12.00           11.06            26.00
 44       2.00           15.00            31.56)

## Again Iteration Process

In [27]:
df2 = df1.copy()
df2

,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.14,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.06,26.00
44,2.00,15.00,31.56


In [28]:
df2.iloc[1, 0] = np.NaN
X = df2.iloc[[0,2,3,4], [1,2]]
Y = df2.iloc[[0,2,3,4], [0]]
lr = LinearRegression()
lr.fit(X,Y)
pred = lr.predict(df2.iloc[1,1:].values.reshape(1,2))
df2.iloc[1, 0] = pred.round(2)
df2

C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.79,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.06,26.00
44,2.00,15.00,31.56


In [29]:
df2.iloc[3, 1] = np.NaN
X = df2.iloc[[0,1,2,4], [0,2]]
Y = df2.iloc[[0,1,2,4], 1]
lr.fit(X,Y)
pred = lr.predict(df2.iloc[3, [0,2]].values.reshape(1,2)).round(2)
print(pred)
df2.iloc[3,1] = pred
df2

[11.22]


C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.79,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.22,26.00
44,2.00,15.00,31.56


In [30]:
df2.iloc[-1, -1] = np.NaN
X = df2.iloc[:-1, 0:2]
Y = df2.iloc[:-1, -1]
lr.fit(X,Y)
pred = lr.predict(df2.iloc[-1, 0:2].values.reshape(1,2)).round(2)
print(pred)
df2.iloc[-1, -1] = pred
df2

[38.94]


C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,23.79,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.22,26.00
44,2.00,15.00,38.94


In [31]:
df2 - df1

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.00
37,0.65,0.00,0.00
2,0.00,0.00,0.00
14,0.00,0.16,0.00
44,0.00,0.00,7.38


## Again Iteration Process

In [32]:
df3 = df2.copy()

In [33]:
df3.iloc[1, 0] = np.NaN
X = df3.iloc[[0,2,3,4], [1,2]]
Y = df3.iloc[[0,2,3,4], [0]]
lr = LinearRegression()
lr.fit(X,Y)
pred = lr.predict(df3.iloc[1,1:].values.reshape(1,2))
print(pred)
df3.iloc[1, 0] = pred.round(2)
df3

C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


[[26.8247132]]


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,26.82,5.00,20.00
2,15.00,10.00,41.00
14,12.00,11.22,26.00
44,2.00,15.00,38.94


In [34]:
df3.iloc[3, 1] = np.NaN
X = df3.iloc[[0,1,2,4], [0,2]]
Y = df3.iloc[[0,1,2,4], 1]
lr.fit(X,Y)
pred = lr.predict(df3.iloc[3, [0,2]].values.reshape(1,2)).round(2)
print(pred)
df3.iloc[3,1] = pred
df3

[12.23]


C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,26.82,5.00,20.00
2,15.00,10.00,41.00
14,12.00,12.23,26.00
44,2.00,15.00,38.94


In [35]:
df3.iloc[-1, -1] = np.NaN
X = df3.iloc[:-1, 0:2]
Y = df3.iloc[:-1, -1]
lr.fit(X,Y)
pred = lr.predict(df3.iloc[-1, 0:2].values.reshape(1,2)).round(2)
print(pred)
df3.iloc[-1, -1] = pred
df3

[62.88]


C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,26.82,5.00,20.00
2,15.00,10.00,41.00
14,12.00,12.23,26.00
44,2.00,15.00,62.88


In [36]:
df3-df2

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.00
37,3.03,0.00,0.00
2,0.00,0.00,0.00
14,0.00,1.01,0.00
44,0.00,0.00,23.94


### Again

In [37]:
df4 = df3.copy()
df4.iloc[1, 0] = np.NaN
X = df4.iloc[[0,2,3,4], [1,2]]
Y = df4.iloc[[0,2,3,4], [0]]
lr = LinearRegression()
lr.fit(X,Y)
pred = lr.predict(df4.iloc[1,1:].values.reshape(1,2))
print(pred)
df4.iloc[1, 0] = pred.round(2)
df4

[[26.62764922]]


C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,26.63,5.00,20.00
2,15.00,10.00,41.00
14,12.00,12.23,26.00
44,2.00,15.00,62.88


In [38]:
df4.iloc[3, 1] = np.NaN
X = df4.iloc[[0,1,2,4], [0,2]]
Y = df4.iloc[[0,1,2,4], 1]
lr.fit(X,Y)
pred = lr.predict(df4.iloc[3, [0,2]].values.reshape(1,2)).round(2)
print(pred)
df4.iloc[3,1] = pred
df4

[13.06]


C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,26.63,5.00,20.00
2,15.00,10.00,41.00
14,12.00,13.06,26.00
44,2.00,15.00,62.88


In [39]:
df4.iloc[-1, -1] = np.NaN
X = df4.iloc[:-1, 0:2]
Y = df4.iloc[:-1, -1]
lr.fit(X,Y)
pred = lr.predict(df4.iloc[-1, 0:2].values.reshape(1,2)).round(2)
print(pred)
df4.iloc[-1, -1] = pred
df4

[71.13]


C:\Users\Mohd Rizwan\Python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,R&D Spend,Administration,Marketing Spend
21,8.00,15.00,30.00
37,26.63,5.00,20.00
2,15.00,10.00,41.00
14,12.00,13.06,26.00
44,2.00,15.00,71.13


In [40]:
df4 - df3

,R&D Spend,Administration,Marketing Spend
21,0.00,0.00,0.00
37,-0.19,0.00,0.00
2,0.00,0.00,0.00
14,0.00,0.83,0.00
44,0.00,0.00,8.25
